In [ ]:
#!/usr/bin/env python3
"""
Jupyter-friendly Schwarzschild kink runs with gamma defined in tortoise coordinates.

Interpretation of the input gamma:
    u = dr*/dt on the initial slice at the wall center,
    gamma = 1/sqrt(1-u^2).

Initial data use the local Schwarzschild/tortoise boosted kink centered at r*_0,
with the redshift evaluated at the wall center:

    sf0 = sqrt(f(r*_0))
    xi(0,r*) = gamma * sf0 * (r* - r*_0)
    phi(0,r*) = v tanh(alpha xi)
    pi(0,r*)  = d_t phi|_{t=0}
               = - v alpha sech^2(alpha xi) * gamma * u * sf0

This makes the initial profile symmetric in r* about r*_0 and makes the input
gamma correspond directly to the coordinate speed in (t, r*).

Resolution is based on the initial local wall thickness in r*:

    delta_local = 1 / (alpha gamma sf0)
    dx <= delta_local / pts_across_eff,

with pts_across_eff = max(pts_across, min_pts_across).

Each case can optionally override rstar0 and gamma.
"""

from __future__ import annotations

import argparse
import csv
import re
from dataclasses import dataclass, replace
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

import numpy as np
import matplotlib


def _running_in_jupyter() -> bool:
    try:
        from IPython import get_ipython  # type: ignore
        ip = get_ipython()
        if ip is None:
            return False
        return ip.__class__.__name__ in {"ZMQInteractiveShell", "Shell"}
    except Exception:
        return False


IN_JUPYTER = _running_in_jupyter()
if not IN_JUPYTER:
    matplotlib.use("Agg")

import matplotlib.pyplot as plt

try:
    import imageio.v2 as imageio
except Exception:  # pragma: no cover
    imageio = None


@dataclass(frozen=True)
class CaseSpec:
    lam: float
    rs: float
    tmax: float
    gamma: Optional[float] = None
    label: str = ""
    rstar0: Optional[float] = None


@dataclass(frozen=True)
class SweepConfig:
    gamma: float = 300.0
    v: float = 1.0
    rstar0: float = -20.0
    rstar_min: float = -25.0
    rstar_max: float = 40.0
    pts_across: float = 30.0
    min_pts_across: float = 10.0
    min_domain_points: int = 2001
    courant: float = 0.25
    default_tmax: float = 30.0
    cases: Tuple[CaseSpec, ...] = (
        CaseSpec(lam=1 / 10, rs=6.0, tmax=60.0, gamma=400.0, label="run_example_1", rstar0=-20.0),
        CaseSpec(lam=5 / 10, rs=6.0, tmax=60.0, gamma=25/0.11384, label="run_example_2", rstar0=-20.0)
    )
    make_gif: bool = False
    n_gif_frames: int = 160
    n_profile_snapshots: int = 32
    thickness_lo: float = -0.7
    thickness_hi: float = 0.7
    core_lo: float = -0.8
    core_hi: float = 0.8
    out_dir: Path = Path("out_bh_cases_tortoise_gamma")


@dataclass(frozen=True)
class RunMetadata:
    label: str
    lam: float
    rs: float
    M: float
    alpha: float
    gamma: float
    u: float
    rstar0: float
    r0: float
    f0_exact: float
    sf0_exact: float
    delta_local: float
    dx: float
    dt: float
    nsteps: int
    rstar_min: float
    rstar_max: float
    tmax: float


CONFIG = SweepConfig()


# -----------------------------------------------------------------------------
# Geometry
# -----------------------------------------------------------------------------
def r_of_rstar(rstar: np.ndarray, M: float) -> np.ndarray:
    rstar = np.asarray(rstar, dtype=float)
    r = np.empty_like(rstar)

    far = rstar >= 10.0 * M
    near = ~far

    r[far] = rstar[far] + 2.0 * M
    eps = np.exp((rstar[near] - 2.0 * M) / (2.0 * M))
    r[near] = 2.0 * M * (1.0 + np.maximum(eps, 1e-14))

    rs = 2.0 * M
    for _ in range(80):
        r = np.maximum(r, rs * (1.0 + 1e-14))
        y = np.maximum(r / rs - 1.0, 1e-300)
        F = r + rs * np.log(y) - rstar
        dF = r / (r - rs)
        dr = -F / dF
        r = r + dr
        if np.max(np.abs(dr)) < 1e-13:
            break

    return np.maximum(r, rs * (1.0 + 1e-14))


def first_derivative(y: np.ndarray, dx: float) -> np.ndarray:
    dy = np.empty_like(y)
    dy[1:-1] = (y[2:] - y[:-2]) / (2.0 * dx)
    dy[0] = (-3.0 * y[0] + 4.0 * y[1] - y[2]) / (2.0 * dx)
    dy[-1] = (3.0 * y[-1] - 4.0 * y[-2] + y[-3]) / (2.0 * dx)
    return dy


def second_derivative(y: np.ndarray, dx: float) -> np.ndarray:
    d2y = np.empty_like(y)
    d2y[1:-1] = (y[2:] - 2.0 * y[1:-1] + y[:-2]) / (dx * dx)
    d2y[0] = (2.0 * y[0] - 5.0 * y[1] + 4.0 * y[2] - y[3]) / (dx * dx)
    d2y[-1] = (2.0 * y[-1] - 5.0 * y[-2] + 4.0 * y[-3] - y[-4]) / (dx * dx)
    return d2y


# -----------------------------------------------------------------------------
# Potential and initializer helpers
# -----------------------------------------------------------------------------
def V(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return 0.25 * lam * (phi * phi - v * v) ** 2


def dV_dphi(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return lam * phi * (phi * phi - v * v)


def stable_sech2(z: np.ndarray) -> np.ndarray:
    az = np.abs(z)
    out = np.empty_like(z)
    mask = az < 40.0
    out[mask] = 1.0 / np.cosh(z[mask]) ** 2
    out[~mask] = 4.0 * np.exp(-2.0 * az[~mask])
    return out


def resolved_case_tmax(case: CaseSpec, cfg: SweepConfig) -> float:
    tmax = float(case.tmax if case.tmax is not None else cfg.default_tmax)
    if not np.isfinite(tmax) or tmax <= 0.0:
        raise ValueError(f"Invalid tmax={tmax} for case {case!r}.")
    return tmax


def resolved_case_rstar0(case: CaseSpec, cfg: SweepConfig) -> float:
    rstar0 = float(cfg.rstar0 if case.rstar0 is None else case.rstar0)
    if not np.isfinite(rstar0):
        raise ValueError(f"Invalid rstar0={rstar0} for case {case!r}.")
    return rstar0


def resolved_case_gamma(case: CaseSpec, cfg: SweepConfig) -> float:
    gamma = float(cfg.gamma if case.gamma is None else case.gamma)
    if not np.isfinite(gamma) or gamma < 1.0:
        raise ValueError(f"Invalid gamma={gamma} for case {case!r}. Gamma must satisfy gamma >= 1.")
    return gamma


def sanitize_label(label: str) -> str:
    label = re.sub(r"[^A-Za-z0-9_.-]+", "_", label.strip())
    label = re.sub(r"_+", "_", label)
    return label.strip("_.") or "case"


def unique_case_labels(cases: Sequence[CaseSpec], cfg: SweepConfig) -> List[str]:
    raw = []
    for case in cases:
        if case.label.strip():
            raw.append(sanitize_label(case.label))
        else:
            raw.append(
                sanitize_label(
                    f"lam_{case.lam:.8g}__rs_{case.rs:.8g}"
                    f"__gamma_{resolved_case_gamma(case, cfg):.8g}"
                    f"__rstar0_{resolved_case_rstar0(case, cfg):.8g}"
                    f"__tmax_{resolved_case_tmax(case, cfg):.8g}"
                )
            )

    seen: Dict[str, int] = {}
    out: List[str] = []
    duplicates = {label for label in raw if raw.count(label) > 1}
    for label in raw:
        seen[label] = seen.get(label, 0) + 1
        if label not in duplicates:
            out.append(label)
        else:
            out.append(f"{label}__case{seen[label]:02d}")
    return out


# -----------------------------------------------------------------------------
# Diagnostics helpers
# -----------------------------------------------------------------------------
def level_crossings(x: np.ndarray, phi: np.ndarray, level: float) -> np.ndarray:
    y = phi - level
    candidates: List[float] = []

    exact = np.where(np.isclose(y, 0.0, rtol=0.0, atol=1e-14))[0]
    if exact.size:
        candidates.extend(x[exact].tolist())

    prod = y[:-1] * y[1:]
    idx = np.where(prod < 0.0)[0]
    for i in idx:
        x0, x1 = x[i], x[i + 1]
        y0, y1 = y[i], y[i + 1]
        xc = x0 - y0 * (x1 - x0) / (y1 - y0)
        candidates.append(float(xc))

    if not candidates:
        return np.empty(0, dtype=float)
    return np.array(sorted(set(candidates)), dtype=float)


def rightmost_level_crossing(x: np.ndarray, phi: np.ndarray, level: float) -> float:
    roots = level_crossings(x, phi, level)
    if roots.size == 0:
        return float("nan")
    return float(roots[-1])


def kink_thickness_rstar(phi: np.ndarray, x: np.ndarray, lo: float, hi: float) -> float:
    r_hi = rightmost_level_crossing(x, phi, hi)
    r_lo = rightmost_level_crossing(x, phi, lo)
    if not (np.isfinite(r_hi) and np.isfinite(r_lo)):
        return float("nan")
    return float(r_hi - r_lo)


# -----------------------------------------------------------------------------
# Output helpers
# -----------------------------------------------------------------------------
def ensure_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)


def write_run_metadata(run_dir: Path, meta: RunMetadata) -> None:
    with (run_dir / "run_metadata.txt").open("w", encoding="utf-8") as f:
        for key, value in meta.__dict__.items():
            f.write(f"{key}={value}\n")


def write_phi_profiles(path: Path, x: np.ndarray, snapshots: List[Tuple[float, np.ndarray]]) -> None:
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["t", "rstar", "phi"])
        for t, phi in snapshots:
            for xi, phii in zip(x, phi):
                writer.writerow([f"{t:.16e}", f"{xi:.16e}", f"{phii:.16e}"])


def save_diagnostic_csv(path: Path, times: np.ndarray, kpos: np.ndarray, thk: np.ndarray, Ek: np.ndarray, Et: np.ndarray) -> None:
    e0 = Et[0]
    np.savetxt(
        path,
        np.column_stack([times, kpos, thk, Ek, Et, Et / e0, Et - e0]),
        delimiter=",",
        header=(
            "t,"
            "rstar_kink_rightmost_phi0,"
            "thickness_rstar(phi=+0.7 minus phi=-0.7),"
            "Ekink_core(-0.8<phi<0.8),"
            "Etot,"
            "Etot_over_E0,"
            "DeltaE"
        ),
        comments="",
    )


def save_initial_plots(run_dir: Path, x: np.ndarray, phi0: np.ndarray, meta: RunMetadata) -> None:
    fig = plt.figure(figsize=(7.0, 4.0), dpi=120)
    plt.plot(x, phi0, lw=1.2)
    plt.axhline(0.0, lw=0.8)
    plt.axvline(meta.rstar0, lw=0.8, ls="--")
    plt.xlim(meta.rstar_min, meta.rstar_max)
    plt.ylim(-1.25, 1.25)
    plt.xlabel("r*")
    plt.ylabel("phi")
    plt.title(f"Initial phi(r*) | {meta.label}")
    plt.tight_layout()
    plt.savefig(run_dir / "initial_phi.png")
    plt.close(fig)

    half_window = max(2.0, 6.0 * meta.delta_local)
    fig = plt.figure(figsize=(7.0, 4.0), dpi=120)
    plt.plot(x, phi0, lw=1.2)
    plt.axhline(0.0, lw=0.8)
    plt.axvline(meta.rstar0, lw=0.8, ls="--")
    plt.xlim(meta.rstar0 - half_window, meta.rstar0 + half_window)
    plt.ylim(-1.25, 1.25)
    plt.xlabel("r*")
    plt.ylabel("phi")
    plt.title(f"Initial phi(r*) zoom | {meta.label}")
    plt.tight_layout()
    plt.savefig(run_dir / "initial_phi_zoom.png")
    plt.close(fig)


def save_run_plots(
    run_dir: Path,
    meta: RunMetadata,
    x: np.ndarray,
    phi_final: np.ndarray,
    times: np.ndarray,
    kpos: np.ndarray,
    thk: np.ndarray,
    Ek: np.ndarray,
    Et: np.ndarray,
) -> None:
    e0 = Et[0]

    fig, axs = plt.subplots(4, 1, figsize=(7.6, 10.0), dpi=120, sharex=True)
    axs[0].plot(times, kpos, lw=1.3)
    axs[0].set_ylabel("r*_kink")
    axs[0].set_title(f"Kink diagnostics vs time ({meta.label})")

    axs[1].plot(times, thk, lw=1.3)
    axs[1].set_ylabel("Delta r*")

    axs[2].plot(times, Ek, lw=1.3)
    axs[2].set_ylabel("Ekink core")

    axs[3].plot(times, Et / e0, lw=1.3)
    axs[3].set_ylabel("Etot / E0")
    axs[3].set_xlabel("t")

    fig.tight_layout()
    fig.savefig(run_dir / "kink_diagnostics.png")
    plt.close(fig)

    fig, ax = plt.subplots(2, 1, figsize=(7.6, 6.6), dpi=120, sharex=True)
    ax[0].plot(times, Et, lw=1.3)
    ax[0].set_ylabel("Etot")
    ax[0].set_title(f"Energy check on finite domain ({meta.label})")

    ax[1].plot(times, Et - e0, lw=1.3)
    ax[1].set_ylabel("Etot - E0")
    ax[1].set_xlabel("t")

    fig.tight_layout()
    fig.savefig(run_dir / "energy_conservation.png")
    plt.close(fig)

    fig = plt.figure(figsize=(7.0, 4.0), dpi=120)
    plt.plot(x, phi_final, lw=1.2)
    plt.axhline(0.0, lw=0.8)
    plt.xlim(meta.rstar_min, meta.rstar_max)
    plt.ylim(-1.25, 1.25)
    plt.xlabel("r*")
    plt.ylabel("phi")
    plt.title(f"Final phi(r*) at t={meta.tmax:.3f} | {meta.label}")
    plt.tight_layout()
    plt.savefig(run_dir / "final_phi.png")
    plt.close(fig)


# -----------------------------------------------------------------------------
# Single-case setup and evolution
# -----------------------------------------------------------------------------
def build_case_geometry(case: CaseSpec, cfg: SweepConfig, resolved_label: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, RunMetadata]:
    alpha = cfg.v * np.sqrt(case.lam / 2.0)
    case_gamma = resolved_case_gamma(case, cfg)
    u = np.sqrt(max(0.0, 1.0 - case_gamma ** -2))

    case_rstar0 = resolved_case_rstar0(case, cfg)
    M = case.rs / 2.0
    r0 = float(r_of_rstar(np.array([case_rstar0], dtype=float), M)[0])
    f0_exact = max(1.0 - case.rs / r0, 1e-300)
    sf0_exact = float(np.sqrt(f0_exact))
    delta_local = 1.0 / (alpha * case_gamma * sf0_exact)

    pts_across_eff = max(float(cfg.pts_across), float(cfg.min_pts_across))
    dx_from_wall = delta_local / pts_across_eff

    domain_length = float(cfg.rstar_max - cfg.rstar_min)
    if domain_length <= 0.0:
        raise ValueError("Require rstar_max > rstar_min.")
    min_domain_points = max(int(cfg.min_domain_points), 5)
    dx_from_domain = domain_length / float(min_domain_points - 1)
    dx = min(dx_from_wall, dx_from_domain)

    x = np.arange(cfg.rstar_min, cfg.rstar_max + 0.5 * dx, dx, dtype=float)
    if x.size < 5:
        raise ValueError("Need at least 5 grid points; enlarge the domain or decrease dx.")

    r = r_of_rstar(x, M)
    f = 1.0 - case.rs / r
    geom_factor = 2.0 * f / r

    case_tmax = resolved_case_tmax(case, cfg)
    dt_nominal = cfg.courant * dx
    nsteps = max(int(np.ceil(case_tmax / dt_nominal)), 1)
    dt = case_tmax / nsteps

    meta = RunMetadata(
        label=resolved_label,
        lam=case.lam,
        rs=case.rs,
        M=M,
        alpha=alpha,
        gamma=case_gamma,
        u=u,
        rstar0=case_rstar0,
        r0=r0,
        f0_exact=f0_exact,
        sf0_exact=sf0_exact,
        delta_local=delta_local,
        dx=dx,
        dt=dt,
        nsteps=nsteps,
        rstar_min=cfg.rstar_min,
        rstar_max=cfg.rstar_max,
        tmax=case_tmax,
    )
    return x, r, f, geom_factor, meta


def build_initial_data(x: np.ndarray, meta: RunMetadata, cfg: SweepConfig) -> Tuple[np.ndarray, np.ndarray]:
    xi0 = meta.gamma * meta.sf0_exact * (x - meta.rstar0)
    arg = meta.alpha * xi0
    phi = cfg.v * np.tanh(arg)
    pi = -cfg.v * meta.alpha * stable_sech2(arg) * (meta.gamma * meta.u * meta.sf0_exact)
    return phi, pi


def apply_sommerfeld_bc(phi: np.ndarray, pi: np.ndarray, dx: float) -> None:
    phi_x_L = (-3.0 * phi[0] + 4.0 * phi[1] - phi[2]) / (2.0 * dx)
    phi_x_R = (3.0 * phi[-1] - 4.0 * phi[-2] + phi[-3]) / (2.0 * dx)
    pi[0] = +phi_x_L
    pi[-1] = -phi_x_R


def rhs(phi: np.ndarray, pi: np.ndarray, dx: float, r: np.ndarray, f: np.ndarray, geom_factor: np.ndarray, lam: float, v: float) -> Tuple[np.ndarray, np.ndarray]:
    phi_x = first_derivative(phi, dx)
    phi_xx = second_derivative(phi, dx)
    pi_t = phi_xx + geom_factor * phi_x - f * dV_dphi(phi, lam, v)
    return pi, pi_t


def rk4_step(phi: np.ndarray, pi: np.ndarray, dt: float, dx: float, r: np.ndarray, f: np.ndarray, geom_factor: np.ndarray, lam: float, v: float) -> Tuple[np.ndarray, np.ndarray]:
    apply_sommerfeld_bc(phi, pi, dx)
    k1_phi, k1_pi = rhs(phi, pi, dx, r, f, geom_factor, lam, v)

    phi2 = phi + 0.5 * dt * k1_phi
    pi2 = pi + 0.5 * dt * k1_pi
    apply_sommerfeld_bc(phi2, pi2, dx)
    k2_phi, k2_pi = rhs(phi2, pi2, dx, r, f, geom_factor, lam, v)

    phi3 = phi + 0.5 * dt * k2_phi
    pi3 = pi + 0.5 * dt * k2_pi
    apply_sommerfeld_bc(phi3, pi3, dx)
    k3_phi, k3_pi = rhs(phi3, pi3, dx, r, f, geom_factor, lam, v)

    phi4 = phi + dt * k3_phi
    pi4 = pi + dt * k3_pi
    apply_sommerfeld_bc(phi4, pi4, dx)
    k4_phi, k4_pi = rhs(phi4, pi4, dx, r, f, geom_factor, lam, v)

    phi_new = phi + (dt / 6.0) * (k1_phi + 2.0 * k2_phi + 2.0 * k3_phi + k4_phi)
    pi_new = pi + (dt / 6.0) * (k1_pi + 2.0 * k2_pi + 2.0 * k3_pi + k4_pi)
    apply_sommerfeld_bc(phi_new, pi_new, dx)
    return phi_new, pi_new


def energy_density_per_drstar(phi: np.ndarray, pi: np.ndarray, dx: float, r: np.ndarray, f: np.ndarray, lam: float, v: float) -> np.ndarray:
    phi_rstar = first_derivative(phi, dx)
    return 4.0 * np.pi * r * r * (0.5 * (pi * pi + phi_rstar * phi_rstar) + f * V(phi, lam, v))


def total_energy(phi: np.ndarray, pi: np.ndarray, dx: float, r: np.ndarray, f: np.ndarray, lam: float, v: float) -> float:
    return float(np.sum(energy_density_per_drstar(phi, pi, dx, r, f, lam, v)) * dx)


def kink_core_energy(phi: np.ndarray, pi: np.ndarray, dx: float, r: np.ndarray, f: np.ndarray, lam: float, v: float, phi_min: float, phi_max: float) -> float:
    mask = (phi > phi_min) & (phi < phi_max)
    e = energy_density_per_drstar(phi, pi, dx, r, f, lam, v)
    return float(np.sum(e[mask]) * dx)


def run_single_case(case: CaseSpec, cfg: SweepConfig, resolved_label: str) -> Dict[str, np.ndarray | float | RunMetadata]:
    x, r, f, geom_factor, meta = build_case_geometry(case, cfg, resolved_label)
    phi, pi = build_initial_data(x, meta, cfg)
    apply_sommerfeld_bc(phi, pi, meta.dx)

    run_dir = cfg.out_dir / resolved_label
    ensure_dir(run_dir)
    write_run_metadata(run_dir, meta)
    save_initial_plots(run_dir, x, phi, meta)

    init_zero = rightmost_level_crossing(x, phi, 0.0)
    print(
        f"Running {resolved_label}: lambda={case.lam:.12g}, rs={case.rs:.12g}, gamma={meta.gamma:.12g}, "
        f"rstar0={meta.rstar0:.12g}, tmax={meta.tmax:.12g}, delta_local={meta.delta_local:.12g}, "
        f"dx={meta.dx:.12g}, dt={meta.dt:.12g}, steps={meta.nsteps}, N={x.size}, init_zero={init_zero:.12g}"
    )
    if meta.nsteps > 2_000_000 or x.size > 500_000:
        print("  Warning: this run is very large and may be slow in Jupyter.")

    nsteps = meta.nsteps
    times = np.zeros(nsteps + 1)
    kpos = np.zeros(nsteps + 1)
    thk = np.zeros(nsteps + 1)
    Ek = np.zeros(nsteps + 1)
    Et = np.zeros(nsteps + 1)

    snapshots: List[Tuple[float, np.ndarray]] = []
    profile_stride = max(1, nsteps // max(1, cfg.n_profile_snapshots))

    writer = None
    fig = None
    line = None
    title = None
    gif_stride = max(1, nsteps // max(1, cfg.n_gif_frames))

    if cfg.make_gif and imageio is not None:
        writer = imageio.get_writer(
            run_dir / "bh_kink_phi.gif",
            mode="I",
            duration=0.05,
            palettesize=64,
            subrectangles=True,
        )
        fig, ax = plt.subplots(figsize=(7.0, 3.8), dpi=90)
        (line,) = ax.plot(x, phi, lw=1.1)
        ax.set_xlim(cfg.rstar_min, cfg.rstar_max)
        ax.set_ylim(-1.25, 1.25)
        ax.set_xlabel("r*")
        ax.set_ylabel("phi")
        title = ax.set_title("")
        ax.axhline(0.0, lw=0.8)
        fig.canvas.draw()
    elif cfg.make_gif and imageio is None:
        print("  imageio is not available; GIF output disabled for this run.")

    def grab_rgb(current_fig: plt.Figure) -> np.ndarray:
        current_fig.canvas.draw()
        rgba = np.asarray(current_fig.canvas.buffer_rgba())
        return rgba[..., :3].copy()

    t = 0.0
    for n in range(nsteps + 1):
        times[n] = t
        kpos[n] = rightmost_level_crossing(x, phi, 0.0)
        thk[n] = kink_thickness_rstar(phi, x, cfg.thickness_lo, cfg.thickness_hi)
        Ek[n] = kink_core_energy(phi, pi, meta.dx, r, f, case.lam, cfg.v, cfg.core_lo, cfg.core_hi)
        Et[n] = total_energy(phi, pi, meta.dx, r, f, case.lam, cfg.v)

        if (n % profile_stride == 0) or (n == nsteps):
            snapshots.append((t, phi.copy()))

        if writer is not None and fig is not None and line is not None and title is not None:
            if (n % gif_stride == 0) or (n == nsteps):
                line.set_ydata(phi)
                title.set_text(
                    f"{resolved_label} | t={t:.3f} | r*_k={kpos[n]:.3f} | "
                    f"Delta r*={thk[n]:.3e} | Et/Et0={Et[n] / Et[0]:.6f}"
                )
                writer.append_data(grab_rgb(fig))

        if n == nsteps:
            break

        phi, pi = rk4_step(phi, pi, meta.dt, meta.dx, r, f, geom_factor, case.lam, cfg.v)
        t += meta.dt

        if not (np.isfinite(phi).all() and np.isfinite(pi).all()):
            raise RuntimeError(f"Non-finite values encountered for {resolved_label} at step {n}, t={t:.16e}.")

    if writer is not None:
        writer.close()
    if fig is not None:
        plt.close(fig)

    save_diagnostic_csv(run_dir / "kink_diagnostics.csv", times, kpos, thk, Ek, Et)
    save_run_plots(run_dir, meta, x, phi, times, kpos, thk, Ek, Et)
    write_phi_profiles(run_dir / "phi_profiles.csv", x, snapshots)

    summary = {
        "label": resolved_label,
        "lambda": case.lam,
        "rs": case.rs,
        "gamma": meta.gamma,
        "tmax": meta.tmax,
        "rstar0": meta.rstar0,
        "alpha": meta.alpha,
        "u": meta.u,
        "f0_exact": meta.f0_exact,
        "r0": meta.r0,
        "delta_local": meta.delta_local,
        "dx": meta.dx,
        "dt": meta.dt,
        "nsteps": float(meta.nsteps),
        "kpos0": kpos[0],
        "kpos_final": kpos[-1],
        "thickness0": thk[0],
        "thickness_final": thk[-1],
        "thickness_min": float(np.nanmin(thk)),
        "thickness_max": float(np.nanmax(thk)),
        "Ek0": Ek[0],
        "Ek_final": Ek[-1],
        "Et0": Et[0],
        "Et_final": Et[-1],
        "Et_ratio_final": Et[-1] / Et[0],
        "Et_ratio_min": float(np.nanmin(Et / Et[0])),
        "Et_ratio_max": float(np.nanmax(Et / Et[0])),
    }

    return {
        "meta": meta,
        "times": times,
        "kpos": kpos,
        "thk": thk,
        "Ek": Ek,
        "Et": Et,
        "summary": summary,
    }


# -----------------------------------------------------------------------------
# Sweep outputs
# -----------------------------------------------------------------------------
def save_summary_csv(path: Path, summaries: List[Dict[str, float | str]]) -> None:
    if not summaries:
        return
    fieldnames = list(summaries[0].keys())
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(summaries)


def save_overview_plot(path: Path, results: Dict[str, Dict[str, np.ndarray | float | RunMetadata]]) -> None:
    fig, axs = plt.subplots(3, 1, figsize=(8.0, 10.0), dpi=130, sharex=False)

    for label, result in results.items():
        times = result["times"]  # type: ignore[index]
        kpos = result["kpos"]  # type: ignore[index]
        thk = result["thk"]  # type: ignore[index]
        Et = result["Et"]  # type: ignore[index]
        axs[0].plot(times, kpos, lw=1.2, label=label)
        axs[1].plot(times, thk, lw=1.2, label=label)
        axs[2].plot(times, Et / Et[0], lw=1.2, label=label)

    axs[0].set_ylabel("r*_kink")
    axs[0].set_title("Schwarzschild kink runs (gamma defined in tortoise coordinates)")
    axs[0].legend(fontsize=8)
    axs[1].set_ylabel("Delta r*")
    axs[2].set_ylabel("Etot / E0")
    axs[2].set_xlabel("t")

    fig.tight_layout()
    fig.savefig(path)
    plt.close(fig)


# -----------------------------------------------------------------------------
# Public entry point
# -----------------------------------------------------------------------------
def run_cases(cfg: Optional[SweepConfig] = None, **overrides: object) -> Dict[str, Dict[str, np.ndarray | float | RunMetadata]]:
    if cfg is None:
        cfg = CONFIG
    if overrides:
        cfg = replace(cfg, **overrides)

    ensure_dir(cfg.out_dir)

    print("Run configuration:")
    print(f"  default gamma={cfg.gamma} (interpreted in tortoise coordinates: u = dr*/dt)")
    print(f"  default r*_0={cfg.rstar0}")
    print(f"  domain=[{cfg.rstar_min}, {cfg.rstar_max}]")
    print(f"  pts_across target={cfg.pts_across}, minimum enforced={cfg.min_pts_across}")
    print(f"  min domain points={cfg.min_domain_points}")
    print(f"  GIF output={'on' if cfg.make_gif else 'off'}")
    print(f"  Jupyter mode={'on' if IN_JUPYTER else 'off'}")
    print("  cases:")

    labels = unique_case_labels(cfg.cases, cfg)
    for case, label in zip(cfg.cases, labels):
        print(
            f"    {label}: lambda={case.lam:.12g}, rs={case.rs:.12g}, "
            f"gamma={resolved_case_gamma(case, cfg):.12g}, "
            f"rstar0={resolved_case_rstar0(case, cfg):.12g}, tmax={resolved_case_tmax(case, cfg):.12g}"
        )

    results: Dict[str, Dict[str, np.ndarray | float | RunMetadata]] = {}
    summaries: List[Dict[str, float | str]] = []

    for case, label in zip(cfg.cases, labels):
        result = run_single_case(case, cfg, label)
        results[label] = result
        summaries.append(result["summary"])  # type: ignore[arg-type]

    save_summary_csv(cfg.out_dir / "summary.csv", summaries)
    save_overview_plot(cfg.out_dir / "overview.png", results)

    print(f"Wrote summary: {(cfg.out_dir / 'summary.csv').resolve()}")
    print(f"Wrote overview plot: {(cfg.out_dir / 'overview.png').resolve()}")
    print("Done.")
    return results


# -----------------------------------------------------------------------------
# CLI parsing, notebook-safe
# -----------------------------------------------------------------------------
def parse_args(argv: Optional[Sequence[str]] = None) -> SweepConfig:
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("--gamma", type=float, default=CONFIG.gamma)
    parser.add_argument("--rstar0", type=float, default=CONFIG.rstar0)
    parser.add_argument("--rstar-min", type=float, default=CONFIG.rstar_min)
    parser.add_argument("--rstar-max", type=float, default=CONFIG.rstar_max)
    parser.add_argument("--pts-across", type=float, default=CONFIG.pts_across)
    parser.add_argument("--min-pts-across", type=float, default=CONFIG.min_pts_across)
    parser.add_argument("--min-domain-points", type=int, default=CONFIG.min_domain_points)
    parser.add_argument("--courant", type=float, default=CONFIG.courant)
    parser.add_argument("--default-tmax", type=float, default=CONFIG.default_tmax)
    parser.add_argument("--out-dir", type=Path, default=CONFIG.out_dir)
    parser.add_argument("--make-gif", action="store_true", default=CONFIG.make_gif)
    parser.add_argument("--n-gif-frames", type=int, default=CONFIG.n_gif_frames)
    parser.add_argument("--n-profile-snapshots", type=int, default=CONFIG.n_profile_snapshots)

    if IN_JUPYTER and argv is None:
        args, _unknown = parser.parse_known_args([])
    else:
        args, _unknown = parser.parse_known_args(argv)

    return SweepConfig(
        gamma=args.gamma,
        v=CONFIG.v,
        rstar0=args.rstar0,
        rstar_min=args.rstar_min,
        rstar_max=args.rstar_max,
        pts_across=args.pts_across,
        min_pts_across=args.min_pts_across,
        min_domain_points=args.min_domain_points,
        courant=args.courant,
        default_tmax=args.default_tmax,
        cases=CONFIG.cases,
        make_gif=bool(args.make_gif),
        n_gif_frames=args.n_gif_frames,
        n_profile_snapshots=args.n_profile_snapshots,
        thickness_lo=CONFIG.thickness_lo,
        thickness_hi=CONFIG.thickness_hi,
        core_lo=CONFIG.core_lo,
        core_hi=CONFIG.core_hi,
        out_dir=args.out_dir,
    )


def main(argv: Optional[Sequence[str]] = None) -> Dict[str, Dict[str, np.ndarray | float | RunMetadata]]:
    cfg = parse_args(argv)
    return run_cases(cfg)


if __name__ == "__main__":
    main()


Run configuration:
  default gamma=300.0 (interpreted in tortoise coordinates: u = dr*/dt)
  default r*_0=-20.0
  domain=[-25.0, 40.0]
  pts_across target=30.0, minimum enforced=10.0
  min domain points=2001
  GIF output=off
  Jupyter mode=on
  cases:
    run2: lambda=0.5, rs=6, gamma=219.606465214, rstar0=-20, tmax=60
    run2_c100: lambda=0.5, rs=6, gamma=105.411103303, rstar0=-20, tmax=80
    run2_c50: lambda=0.5, rs=6, gamma=52.7055516514, rstar0=-20, tmax=80
    run2_c25: lambda=0.5, rs=6, gamma=26.3527758257, rstar0=-20, tmax=80
    run2_c12: lambda=0.5, rs=6, gamma=13.1763879129, rstar0=-20, tmax=80
    run2_c6: lambda=0.5, rs=6, gamma=6.58819395643, rstar0=-20, tmax=80
Running run2: lambda=0.5, rs=6, gamma=219.606465214, rstar0=-20, tmax=60, delta_local=0.0805312403552, dx=0.00268437467851, dt=0.000671088393526, steps=89407, N=24215, init_zero=-20.0000000607
Running run2_c100: lambda=0.5, rs=6, gamma=105.411103303, rstar0=-20, tmax=80, delta_local=0.167773417407, dx=0.005592447